# Technosphere flows induced by the REMIND electricity sector

Here we are interested in the flows of a specific (processed) material in the electricity sector in REMIND. **Values are given per kWh**.

In [1]:
%run initialize_notebook.ipynb

In [2]:
import helpers.ei2rmnd as ei2rmnd

Geomatcher: Used 'AU' for 'AUS'
Geomatcher: Used 'CA' for 'CAN'
Geomatcher: Used 'HM' for 'HMD'
Geomatcher: Used 'NZ' for 'NZL'
Geomatcher: Used 'PM' for 'SPM'
Geomatcher: Used 'CN' for 'CHN'
Geomatcher: Used 'HK' for 'HKG'
Geomatcher: Used 'MO' for 'MAC'
Geomatcher: Used 'TW' for 'TWN'
Geomatcher: Used 'AX' for 'ALA'
Geomatcher: Used 'AT' for 'AUT'
Geomatcher: Used 'BE' for 'BEL'
Geomatcher: Used 'BG' for 'BGR'
Geomatcher: Used 'CY' for 'CYP'
Geomatcher: Used 'CZ' for 'CZE'
Geomatcher: Used 'DE' for 'DEU'
Geomatcher: Used 'DK' for 'DNK'
Geomatcher: Used 'ES' for 'ESP'
Geomatcher: Used 'EE' for 'EST'
Geomatcher: Used 'FI' for 'FIN'
Geomatcher: Used 'FR' for 'FRA'
Geomatcher: Used 'FO' for 'FRO'
Geomatcher: Used 'GB' for 'GBR'
Geomatcher: Used 'GI' for 'GIB'
Geomatcher: Used 'GR' for 'GRC'
Geomatcher: Used 'HR' for 'HRV'
Geomatcher: Used 'HU' for 'HUN'
Geomatcher: Used 'IM' for 'IMN'
Geomatcher: Used 'IE' for 'IRL'
Geomatcher: Used 'IT' for 'ITA'
Geomatcher: Used 'LT' for 'LTU'
Geomatch

In [3]:
import helpers.eimod as eimod

In [4]:
databases

Databases dictionary with 9 object(s):
	Carma CCS
	biosphere3
	ecoinvent_3.5
	ecoinvent_Remind_BAU_2030
	ecoinvent_Remind_BAU_2050
	ecoinvent_Remind_RCP26_2030
	ecoinvent_Remind_RCP26_2050
	ecoinvent_Remind_RCP37_2030
	ecoinvent_Remind_RCP37_2050

In [5]:
def rmnd_technoflows(region, scenario="BAU", year=2030, tech_primers={}):
    """ Extract technosphere flows for the REMIND electricity sector in a REMIND region.
        A dict with primers {x: y} can be given to specify ecoinvent technology y to represent
        REMIND technology x.
    """
    carma_techs = ['Biomass IGCC CCS', 'Biomass IGCC', 'Coal IGCC', 'Coal IGCC CCS', 'Coal PC CCS', 'Gas CCS']

    # note that Carma Database is linked to a specific version of ecoinvent!
    eidb_name = ei2rmnd.get_REMIND_database_name(scenario, year)
    eidb = Database(eidb_name)
    carmadb_name = import_karma(eidb_name=eidb_name)
    carmadb = Database(carmadb_name)
    
    result = {}
    bio_names = []
    regions = ei2rmnd.ei_locations_in_remind_region(region)
    
    techno_markets = {
        "steel": [act for act in eidb if "market for steel," in act["name"] and
                "hot rolled" not in act["name"]],
        "concrete": [act for act in eidb if "market for concrete," in act["name"]],
        "copper": [act for act in eidb if "market for copper" == act["name"]],
        "aluminium": [act for act in eidb if "market for aluminium, primary" in act["name"] or
                    "market for aluminium alloy," in act["name"]],
        "gold": [act for act in eidb if "market for gold" in act["name"]]
    }
    
    print("Found following regions for {}: {}".format(region, regions))
    for key in eimod.available_electricity_generating_technologies:
        print("Processing RMND Technology {}".format(key))
        if key in tech_primers:
            techs = [tech_primers[key]]
        else:
            techs = eimod.available_electricity_generating_technologies[key]
        if len(techs) == 0:
            print("No technologies available for {}! Skipping.".format(key))
            continue
            
        units = {
            "kilowatt hour": 1.,
            "megajoule": 1./3.6
        }
        if key in carma_techs:
            lca = ei2rmnd.lca_for_multiple_techs_and_regions(techs, regions, carmadb,
                                                    units_and_conversions=units)
        else:
            lca = ei2rmnd.lca_for_multiple_techs_and_regions(techs, regions, eidb,
                                                    units_and_conversions=units)

        act_indeces = {
            key: [lca.activity_dict[act.key] for act in techno_markets[key]] for key in techno_markets
        }
            
        # flows are aggregated by the resp. material
        result[key] = pd.DataFrame.from_dict({
            "flow": list(techno_markets.keys()),
            "amount": [lca.supply_array[act_indeces[key]].sum() for key in techno_markets]
        }).groupby("flow").agg({"amount": sum})

    return pd.concat(result)

In [6]:
df = rmnd_technoflows("EUR")

Extracted 1 worksheets in 0.34 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.37 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/19/2018 16:21:38
  Finished: 11/19/2018 16:21:39
  Total time elapsed: 00:00:00
  CPU %: 59.50
  Memory %: 1.89
Created database: Carma CCS
Found following regions for EUR: ['EUR', 'SE', 'FI', 'GR', 'GB', 'AX', 'HR', 'IT', 'ES', 'DE', 'PT', 'DK', 'FR', 'IE', 'BALTSO', 'FO', 'EE', 'NL', 'Canary Islands', 'CENTREL', 'CY', 'MT', 'IM', 'LT', 'AT', 'BE', 'BG', 'CZ', 'GI', 'HU', 'LU', 'LV', 'PL', 'RO', 'SI', 'SK', 'RER']
Processing RMND Technology Biomass IGCC CCS
Processing RMND Technology Biomass IGCC
Processing RMND Technology Coal IGCC
Processing RMND Technology Coal IGCC CCS
Processing RMND Technology Coal PC CCS
Processing RMND Technology Gas CCS
Processing RMND Technology Biomass CHP
Processing RMND Technology Coal PC
Processing RMND Technology Coal CHP
Processing RMND Technology Gas OC
Processing RMND Technology Gas CC
Processing RMND Technology Gas CHP
Processing RMND Technology Geothermal
Processing RMND Technology Hydro


In [7]:
df.sample(20)

amount
                 flow                   
Gas CCS          steel      1.288581e-03
Biomass IGCC CCS concrete   2.803050e-06
Coal IGCC        aluminium  2.357896e-05
                 concrete   2.360047e-06
Solar CSP        steel      3.604675e-03
Gas OC           gold       2.185880e-11
Biomass IGCC     concrete   3.414093e-06
Coal CHP         aluminium  2.069178e-05
Biomass IGCC     aluminium  1.138894e-04
Solar CSP        concrete   5.682131e-06
Coal PC CCS      gold       1.643420e-10
Nuclear          gold       4.232258e-11
Biomass IGCC     copper     1.467684e-04
Solar PV         steel      1.933196e-03
Nuclear          steel      4.315302e-04
Gas CHP          steel      8.562356e-04
Gas CC           concrete   2.802490e-07
Coal IGCC CCS    aluminium  2.895790e-05
Gas OC           concrete   5.252113e-07
Biomass CHP      copper     8.300859e-06